In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [17]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.regularizers import l2

In [18]:
df_loan_data = pd.read_csv("loan_data.csv")

In [19]:
# df_loan_data.head()

In [20]:
# df_loan_data.info()

In [21]:
# Separate features and target variable
X = df_loan_data.drop("not.fully.paid", axis=1)
y = df_loan_data["not.fully.paid"]

In [22]:
# Label encoding for categorical variables
label_encoding = preprocessing.LabelEncoder()
# X["purpose"] = label_encoding.fit_transform(X["purpose"])

In [23]:
# One-hot encode categorical features
X = pd.get_dummies(X, columns=["purpose"])

In [9]:
# X.head()

In [24]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [25]:
# Scale numerical features

# scaler = StandardScaler()
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
# Define the model architecture
model = Sequential()
model.add(
    Dense(128, activation="relu", input_shape=(X_train.shape[1],))
)  # kernel_regularizer=l2(0.01),
model.add(
    Dense(
        64,
        activation="relu",
    )
)  # kernel_regularizer=l2(0.01)
model.add(Dense(1, activation="sigmoid"))

In [27]:
# Compile the model
from keras.metrics import Precision, Recall, AUC

model.compile(
    loss="binary_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=["accuracy", Precision(), Recall(), AUC()],
)

In [28]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
240/240 [==============================] - 1s 2ms/step - loss: 0.4343 - accuracy: 0.8395 - precision_1: 0.4375 - recall_1: 0.0057 - auc_1: 0.6081
Epoch 2/20
240/240 [==============================] - 0s 1ms/step - loss: 0.4173 - accuracy: 0.8401 - precision_1: 0.6667 - recall_1: 0.0049 - auc_1: 0.6602
Epoch 3/20
240/240 [==============================] - 0s 1ms/step - loss: 0.4170 - accuracy: 0.8404 - precision_1: 0.6471 - recall_1: 0.0090 - auc_1: 0.6590
Epoch 4/20
240/240 [==============================] - 0s 1ms/step - loss: 0.4149 - accuracy: 0.8414 - precision_1: 0.8095 - recall_1: 0.0138 - auc_1: 0.6658
Epoch 5/20
240/240 [==============================] - 0s 1ms/step - loss: 0.4127 - accuracy: 0.8416 - precision_1: 0.7059 - recall_1: 0.0195 - auc_1: 0.6728
Epoch 6/20
240/240 [==============================] - 0s 1ms/step - loss: 0.4126 - accuracy: 0.8401 - precision_1: 0.5349 - recall_1: 0.0187 - auc_1: 0.6701
Epoch 7/20
240/240 [==============================] - 0s 1

In [29]:
# Evaluate the model on test data
loss, accuracy, precision, recall, auc = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)
print("Test precision:", precision)
print("Test recall:", recall)
print("Test auc:", auc)

60/60 [==============================] - 0s 1ms/step - loss: 0.4076 - accuracy: 0.8372 - precision_1: 0.2667 - recall_1: 0.0131 - auc_1: 0.7032    
Test loss: 0.4076022207736969
Test accuracy: 0.8371607661247253
Test precision: 0.2666666805744171
Test recall: 0.013114754110574722
Test auc: 0.7031687498092651
